In [22]:
import cv2 as cv
import numpy as np
import os 
# 导入 多次感知机的分类模型 MLPClassifier 
from sklearn.neural_network import MLPClassifier
# 导入 完成模型持久化的 joblib
import joblib

In [23]:
# 初始配置
# 训练集、测试集位置
TRAIN_DIR = 'data/enu_train'
TEST_DIR = 'data/enu_test'
# 图片宽、高
IMAGE_WIDTH = 20
IMAGE_HEIGHT = 20
# 给出类别数、类别-数值 字典
CLASSIFICATION_COUNT = 34
LABEL_DICT = {
	'0':0, '1':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9,
	'A':10, 'B':11, 'C':12, 'D':13, 'E':14, 'F':15, 'G':16, 'H':17,	'J':18, 'K':19,
	'L':20, 'M':21, 'N':22, 'P':23, 'Q':24, 'R':25, 'S':26, 'T':27, 'U':28, 'V':29,
	'W':30, 'X':31, 'Y':32, 'Z':33
}
# 模型持久化的位置
MLP_ENU_MODEL_PATH = 'model/mlp_enu.m'

## 训练

In [24]:
# 1. 加载数据集
data = []
labels = []
# 获取数据集目录下的所有的子目录，并逐一遍历
for item in os.listdir(TRAIN_DIR):
    # 获取每一个具体样本类型的 os 的路径形式
    item_path = os.path.join(TRAIN_DIR, item)
    # 判断只有目录，才进入进行下一级目录的遍历
    if os.path.isdir(item_path):
        # 到了每一个样本目录，遍历其下的每一个训练集样本文件-图片
        for subitem in os.listdir(item_path):
            subitem_path = os.path.join(item_path, subitem)
            gray_image = cv.imread(subitem_path, cv.IMREAD_GRAYSCALE)
            resized_image = cv.resize(gray_image, (IMAGE_WIDTH, IMAGE_HEIGHT))
            data.append(resized_image.ravel())
            labels.append(LABEL_DICT[item])

# 分别赋值 训练数据特征、训练标签向量
train_data = np.array(data)
train_labels = np.array(labels)

# 2. 预处理
# 标准化
normalized_data  = (train_data - train_data.mean()) / train_data.max()

In [25]:
# 3. 创建模型
model = MLPClassifier(hidden_layer_sizes=(48, 24), solver='lbfgs', alpha=1e-5, random_state=42)

In [26]:
# 4. 训练模型
model.fit(normalized_data, train_labels)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(48, 24), random_state=42,
              solver='lbfgs')

## 模型持久化

In [27]:
# 模型持久化
joblib.dump(model, MLP_ENU_MODEL_PATH)

['model/mlp_enu.m']

## 测试

In [28]:
# 1. 加载数据集
data = []
labels = []
# 获取数据集目录下的所有的子目录，并逐一遍历
for item in os.listdir(TEST_DIR):
    # 获取每一个具体样本类型的 os 的路径形式
    item_path = os.path.join(TEST_DIR, item)
    # 判断只有目录，才进入进行下一级目录的遍历
    if os.path.isdir(item_path):
        # 到了每一个样本目录，遍历其下的每一个训练集样本文件-图片
        for subitem in os.listdir(item_path):
            subitem_path = os.path.join(item_path, subitem)
            gray_image = cv.imread(subitem_path, cv.IMREAD_GRAYSCALE)
            resized_image = cv.resize(gray_image, (IMAGE_WIDTH, IMAGE_HEIGHT))
            data.append(resized_image.ravel())
            labels.append(LABEL_DICT[item])

# 分别赋值 训练数据特征、训练标签向量
test_data = np.array(data)
test_labels = np.array(labels)

In [29]:
# 2. 预处理标准化
test_normalized_data  = (test_data - test_data.mean()) / test_data.max()

In [31]:
# fitted_model = model
fitted_model = joblib.load(MLP_ENU_MODEL_PATH)
predicts = fitted_model.predict(test_normalized_data)
# 评估
errors = np.count_nonzero(predicts - test_labels)
errors
(len(predicts)-errors) / len(predicts)

46

0.9436274509803921